# Indirect Tax Portfolio – Python Data & Database Work Samples

This portfolio demonstrates my ability to work with databases, handle large data sets and automate indirect tax compliance processes using Python. It includes example code for:

- **Creating and interacting with a relational database** of transactions and VAT rates.
- **Calculating VAT due for multiple EMEA jurisdictions** using real 2025 standard rates.
- **Reconciling payments with computed VAT obligations** to identify discrepancies.
- **Automating the generation of monthly VAT returns** and exporting to Excel.
- **Implementing data validation and process improvements** for accuracy and efficiency.

Please note that the data used here are simulated for demonstration purposes. VAT rates are based on the EU's standard VAT rates for 2025【11829144080681†L190-L237】【11829144080681†L294-L299】.


In [ ]:
import sqlite3
import pandas as pd
from datetime import date

# Configure pandas display
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

### Database design

We will create an SQLite database containing two tables:

- `vat_rates` – stores the standard VAT rate for each EMEA jurisdiction (country code and rate).
- `transactions` – sample invoice‑level data with fields for date, country, category (e.g. goods or digital services), net amount and the applicable VAT rate.

For the reconciliation demonstration we will also create a `payments` table that holds the VAT payments remitted to tax authorities. Each record contains the country, period and payment amount.


**Note:** This cell originally contained code to generate sample transactions for Q1 2025. To keep the notebook concise, the implementation has been moved to the corrected cell below, which creates the SQLite tables and inserts sample data.

# Setup database and insert sample data (corrected version)
conn2 = sqlite3.connect('indirect_tax.db')
cur2 = conn2.cursor()

# Create tables
cur2.execute("""
CREATE TABLE IF NOT EXISTS vat_rates (
    country_code TEXT PRIMARY KEY,
    standard_rate REAL
)
""")

cur2.execute("""
CREATE TABLE IF NOT EXISTS transactions (
    id INTEGER PRIMARY KEY,
    invoice_date TEXT,
    country_code TEXT,
    category TEXT,
    net_amount REAL,
    vat_rate REAL,
    vat_amount REAL,
    FOREIGN KEY(country_code) REFERENCES vat_rates(country_code)
)
""")

# Insert VAT rates data
vat_rates_data2 = [
    ('IT', 22.0),
    ('DE', 19.0),
    ('FR', 20.0),
    ('NL', 21.0),
    ('ES', 21.0),
    ('BE', 21.0)
]
cur2.executemany("INSERT OR REPLACE INTO vat_rates (country_code, standard_rate) VALUES (?, ?)", vat_rates_data2)

# Generate sample transactions
import random
random.seed(42)
transaction_records2 = []
transaction_id = 1
for country, rate in vat_rates_data2:
    for month in [1, 2, 3]:
        for _ in range(4):
            net = random.randint(500, 8000)
            vat_amount = net * rate / 100
            category = random.choice(['goods', 'digital'])
            day = random.randint(1, 28)
            invoice_date = date(2025, month, day).isoformat()
            transaction_records2.append((transaction_id, invoice_date, country, category, net, rate, vat_amount))
            transaction_id += 1

cur2.executemany("""
INSERT INTO transactions (id, invoice_date, country_code, category, net_amount, vat_rate, vat_amount)
VALUES (?, ?, ?, ?, ?, ?, ?)
""", transaction_records2)

conn2.commit()
print(f"Created database with {len(transaction_records2)} sample transactions.")

### Computing VAT liability by country

To calculate the VAT liability for each jurisdiction, we read the transactions from the database into a pandas `DataFrame` and group by country. The summary below shows total net sales, VAT collected and the computed liability for each country in Q1 2025.


In [ ]:
# Summarise VAT per country
import pandas as pd

# Read transactions into DataFrame
transactions_df = pd.read_sql_query("SELECT country_code, net_amount, vat_amount FROM transactions", conn2)

# Group by country to compute totals
summary_df = transactions_df.groupby('country_code').agg(
    total_net_sales=('net_amount', 'sum'),
    total_vat_collected=('vat_amount', 'sum')
)
summary_df['vat_liability'] = summary_df['total_vat_collected']

# Display summary
summary_df = summary_df.sort_index()
summary_df

,total_net_sales,total_vat_collected,vat_liability
country_code,,,
BE,"47,074.00","9,885.54","9,885.54"
DE,"43,076.00","8,184.44","8,184.44"
ES,"63,995.00","13,438.95","13,438.95"
FR,"61,405.00","12,281.00","12,281.00"
IT,"50,832.00","11,183.04","11,183.04"
NL,"52,199.00","10,961.79","10,961.79"


In [ ]:
# Create payments table and simulate payments for reconciliation
cur2.execute("""
CREATE TABLE IF NOT EXISTS payments (
    country_code TEXT,
    period TEXT,
    payment_amount REAL,
    FOREIGN KEY(country_code) REFERENCES vat_rates(country_code)
)
""")

# Simulate Q1 2025 VAT payments (payments may differ slightly from liability)
liabilities = summary_df.reset_index()
payments_records = []
import random
for _, row in liabilities.iterrows():
    paid = row['total_vat_collected'] * random.uniform(0.95, 1.05)  # simulate ±5% variance
    payments_records.append((row['country_code'], '2025-Q1', round(paid, 2)))

cur2.executemany("INSERT INTO payments (country_code, period, payment_amount) VALUES (?, ?, ?)", payments_records)
conn2.commit()

# Load payments and reconcile
payments_df = pd.read_sql_query("SELECT * FROM payments", conn2)
reconciliation_df = liabilities.merge(payments_df, on='country_code')
reconciliation_df['difference'] = reconciliation_df['payment_amount'] - reconciliation_df['total_vat_collected']

# Display reconciliation results
reconciliation_df[['country_code', 'total_vat_collected', 'payment_amount', 'difference']]

,country_code,total_vat_collected,payment_amount,difference
0,BE,"9,885.54","10,253.71",368.17
1,BE,"9,885.54","9,699.41",-186.13
2,DE,"8,184.44","8,121.53",-62.91
3,DE,"8,184.44","8,589.69",405.25
4,ES,"13,438.95","13,051.64",-387.31
5,ES,"13,438.95","13,640.37",201.42
6,FR,"12,281.00","12,329.26",48.26
7,FR,"12,281.00","12,204.98",-76.02
8,IT,"11,183.04","11,440.17",257.13
9,IT,"11,183.04","11,202.70",19.66


In [ ]:
# Export VAT summary and reconciliation to an Excel file
report_path = 'vat_q1_2025_report.xlsx'
with pd.ExcelWriter(report_path, engine='openpyxl') as writer:
    summary_df.to_excel(writer, sheet_name='VAT_Summary')
    reconciliation_df.to_excel(writer, sheet_name='Reconciliation', index=False)

print(f"Report exported to {report_path}")

Report exported to vat_q1_2025_report.xlsx


In [ ]:
# Data validation: identify potential anomalies
# Reload transactions with VAT rate for validation
transactions_valid_df = pd.read_sql_query("SELECT country_code, net_amount, vat_rate, vat_amount FROM transactions", conn2)

# Compute expected VAT based on net amount and rate
transactions_valid_df['expected_vat'] = transactions_valid_df['net_amount'] * transactions_valid_df['vat_rate'] / 100

# Flag records where VAT amount does not match expected or net amount is negative
anomalies_df = transactions_valid_df[(transactions_valid_df['net_amount'] < 0) |
                                     (abs(transactions_valid_df['vat_amount'] - transactions_valid_df['expected_vat']) > 0.01)]

print(f"Found {len(anomalies_df)} potential anomalies.")
anomalies_df.head()

Found 0 potential anomalies.


,country_code,net_amount,vat_rate,vat_amount,expected_vat


In [ ]:
# Dynamic pivot: VAT collected per country over time
# Load invoice dates along with VAT amounts
full_df = pd.read_sql_query("SELECT invoice_date, country_code, vat_amount FROM transactions", conn2)
full_df['month'] = pd.to_datetime(full_df['invoice_date']).dt.to_period('M')

pivot_table = full_df.pivot_table(index='month', columns='country_code', values='vat_amount', aggfunc='sum')

pivot_table

country_code,BE,DE,ES,FR,IT,NL
month,,,,,,
2025-01,"4,059.30","3,055.96","3,226.44","3,926.40","3,515.38","4,690.77"
2025-02,"2,235.24","2,607.37","4,329.78","4,511.80","2,637.80","3,849.30"
2025-03,"3,591.00","2,521.11","5,882.73","3,842.80","5,029.86","2,421.72"


### Conclusion

These examples illustrate how Python can be used to automate VAT compliance tasks such as data ingestion, calculation of tax liabilities, reconciliation and reporting. In a production environment, the scripts would connect to an ERP (e.g., Workday), pull large transaction data sets, apply country‑specific tax rules and export compliant returns. The approach promotes accuracy, auditability and efficiency, freeing up time for higher‑value advisory work.
